# Zuweisung Themen an Tweets, Vollversion

In [18]:
# loading libs
import pandas as pd
import numpy as np
# import Stemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer
import nltk

## Datensätze laden

ÖRR-Datensatz wird nicht geladen, wurde bereits klassifiziert.

In [6]:
ureg = pd.read_csv("mergerDaten/uregGes.csv", index_col=0)
# regio = pd.read_csv("mergerDaten/regioGes.csv", index_col=0)
# digi = pd.read_csv("mergerDaten/digiGes.csv", index_col=0)
# ges = pd.read_csv("mergerDaten/gesamtMedien.csv")

/tmp/ipykernel_19625/3357490728.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  ureg = pd.read_csv("mergerDaten/uregGes.csv", index_col=0)


In [22]:
ureg.content[0:5]

0    Das ganze Interview mit Markus Söder (CSU) gib...
1    Vor Beginn des #CSU-Parteitags hat der bayrisc...
2    @andreaslahmewd @bengtbergt @markus_soeder @bu...
3    Immer wieder gibt es #Luftalarm in der ukraini...
4    @unattached_man: Vielen Dank, haben wir an Sve...
Name: content, dtype: object

In [8]:
categories_ureg = {
    'verkehrspolitik': ['ticket', 'bahn', 'autos', 'auto', '9euroticket', 'tesla', 'wissing',
                        'fahren', 'verkehrsminister', 'tickets', 'elektroautos'],
    
    'politikAusland': ['usa', 'china', 'trump', 'biden', 'taiwan', 'chinas', 'nordkorea',
                       'kapitol', 'peking', 'washington', 'xi', 'republikaner', 'pelosi',
                       'trumps', 'südkorea', 'kim', 'supreme', 'demokraten',
                       'eu', 'johnson', 'großbritannien', 'britische', 'kommission', 'premier',
                       'truss', 'premierminister', 'ungarn', 'brüssel', 'staaten', 'britischen',
                       'europäische', 'europäischen', 'premierministerin', 'london', 'leyen',
                       'polen', 'sunak',
                       'frankreich', 'macron', 'italien', 'brasilien', 'frankreichs', 'pen', 'draghi',
                       'italiens', 'meloni', 'bolsonaro', 'französische', 'rom', 'parlamentswahl',
                       'französischen', 'präsidentschaftswahl'],
    
    'wirtschaft': ['marktbericht', 'dax', 'börse', 'dowjones', 'inflation', 'ezb', 'anleger', 'zinsen',
                   'bank', 'unternehmen', 'fed', 'aktien', 'leitzins', 'dollar', 'banken', 'notenbank',
                   'porsche', 'börsen', '@nzzwirtschaft', 'zentralbank', 'gold'],
    
    'naherOstenNordafrika': ['türkei', 'iran', 'israel', 'afghanistan', 'taliban', 'mali', 'erdogan',
                             'proteste', 'einsatz', 'kasachstan', 'bundeswehr', 'türkische', 'syrien',
                             'griechenland', 'soldaten', 'saudi', 'gewalt', 'arabien', 'irak'],
    
    'kulturUnterhaltung': ['film', 'serie', 'star', 'schauspieler', 'netflix', 'tatort', 'regisseur',
                           'album', 'schauspielerin', 'leutedestages', 'oscar', 'rtl', 'band', 'musik',
                           'sängerin', 'kino', 'fans', 'konzert', 'geschichte', 'buch', 'roman', 'gespräch',
                           'autor', 'kunst', 'schriftsteller', 'ausstellung', 'holocaust', 'historiker'],
    
    'energiekrise': ['gas', 'habeck', 'öl', 'energie', 'energiekrise', 'wirtschaftsminister', 'akw', 'gazprom',
                     'atomkraft', 'strom', 'erdgas', 'pipeline', 'gaslieferungen', 'uniper', 'embargo',
                     'inflation', 'preise', 'wirtschaft', 'teurer', 'energiepreise',
                     'energie', 'kosten', 'verbraucher'],
    
    'ukrainekrieg': ['ukraine', 'krieg', 'geflüchtete', 'flüchtlinge', 'ukrainer', 'geflüchteten',
                     'ukrainische', 'gewalt', 'flucht', 'solidarität', 'russland', 'putin', 'krieg',
                     'russischen', 'russische', 'russlands', 'sanktionen', 'putins', 'moskau', 'westen',
                     'kreml', 'angriff', 'konflikt', 'ukrainekrieg', 'angriffskrieg', 'lawrow',
                     'ukraine', 'nato', 'waffen', 'kiew', 'russland', 'waffenlieferungen', 'selenskyj',
                     'g7', 'ukraine', 'krieg', 'russland', 'russische', 'kiew', 'ukrainische', 'russischen',
                     'selenskyj', 'ukrainischen', 'mariupol', 'truppen', 'soldaten', 'armee', 'angriff',
                     'ukrainekrieg', 'russen', 'russischer', 'zivilisten'],
    
    'socialMediaInternet': ['twitter', 'musk', 'internet', 'facebook', 'online', 'medien', 'amazon', 'app',
                            'instagram', 'netz', 'apple', 'google', 'daten', 'tesla', 'social',
                            'plattform', 'tiktok'],
    
    'streiksDemos': ['lufthansa', 'chaos', 'flughafen', 'arbeit', 'streik', 'flughäfen', 'personal',
                     'flüge', 'piloten', 'mitarbeiter', 'gewerkschaft', 'probleme', 'urlaub', 'beschäftigten'],
    
    'umweltKlima': ['klimawandel', 'hitze', 'grad', 'wasser', 'hitzewelle', 'forscher',
                    'temperaturen', 'feuer', 'dürre', 'sommer', 'klimakrise', 'unwetter',
                    'sturm', 'waldbrände', 'wetter', 'klima', 'fischsterben'],
    
    'politikWahlen': ['cdu', 'spd', 'afd', 'grünen', 'partei', 'merz', 'schröder', 'fdp', 'ministerpräsident',
                      'wahl', 'politiker', 'söder', 'csu', 'grüne', 'landtagswahl', 'linke', 'parteitag',
                      'linken'],
    
    'politikInland': ['lindner', 'ampel', 'bundestag', 'fdp', 'impfpflicht',
                                'bundesregierung', 'spd', 'bundeswehr', 'finanzministerium', 'union',
                                'entlastungspaket', 'koalition', 'gasumlage', 'debatte', 'grünen',
                                'energiekrise', 'ampelkoalition'],
    
    'covid': ['corona', 'lauterbach', 'omikron', 'coronavirus', 'pandemie', 'rki', 'inzidenz',
              'neuinfektionen', 'impfpflicht', 'gesundheitsminister', 'impfung', 'welle',
              'maskenpflicht', 'covid']
}

In [9]:
stemmer = Stemmer.Stemmer('de')

dfs_ureg = []

for key,values in categories_ureg.items():
    words=pd.DataFrame({'category': key, 'term': values})
    dfs_ureg.append(words)

In [10]:
terms_df_ureg = pd.concat(dfs_ureg)
terms_df_ureg.to_csv('terms_df_ureg.csv', encoding='utf8')

In [11]:
terms_df_ureg

,category,term
0,verkehrspolitik,ticket
1,verkehrspolitik,bahn
2,verkehrspolitik,autos
3,verkehrspolitik,auto
4,verkehrspolitik,9euroticket
...,...,...
9,covid,gesundheitsminister
10,covid,impfung
11,covid,welle
12,covid,maskenpflicht


In [13]:
# stemmer = Stemmer.Stemmer('de')

# # Based on CountVectorizer
# class StemmedCountVectorizer(CountVectorizer):
#     def build_analyzer(self):
#         analyzer = super(StemmedCountVectorizer, self).build_analyzer()
#         return lambda doc: stemmer.stemWords([w for w in analyzer(doc)])

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/fw/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
# Take the 'term' column from our list of terms
term_list_ureg = list(set(terms_df_ureg.term))

german_stop_words = stopwords.words('german')

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = CountVectorizer(binary=True, ngram_range=(2,2), stop_words=german_stop_words)
matrix = vectorizer.fit_transform(ureg.content[0:5])
words_df_ureg = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names_out())

In [21]:
words_df_ureg.T

,0,1,2,3,4
50er jahre,0,0,1,0,0
alltag kinder,0,0,0,1,0
andreaslahmewd bengtbergt,0,0,1,0,0
ballett training,0,0,0,1,0
bayern gehe,0,1,0,0,0
...,...,...,...,...,...
vergackeiern heinz,0,0,1,0,0
vielen dank,0,0,0,0,1
voran bund,0,1,0,0,0
wenig https,0,1,0,0,0


In [65]:
# print(vectorizer.vocabulary_)

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
# X_train, X_test, y_train, y_test = train_test_split()

In [68]:
# testingvectorizer = CountVectorizer(binary=False, vocabulary=term_list_ureg)
# testingstuff = ureg.content.transform(ureg.content)
# testingstuffDF = pd.DataFrame(testingstuff.toarray(),
#                              columns=testingvectorizer.get_feature_names_out)

In [69]:
# Group the terms by category, then loop through each category
for category_name, rows in terms_df_ureg.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Looking at {category_name} with terms {terms}")

    # words_df[terms] gets the columns for 'student', 'educ', and 'teacher'
    # .any(axis=1) sees if any of them are a 1, gives True/False
    # .astype(int) converts True/False to 1/0
    # df[category_name] = will assign that value to df['education']
    ureg[category_name] = words_df_ureg[terms].any(axis=1).astype(int)

Looking at covid with terms ['corona', 'lauterbach', 'omikron', 'coronavirus', 'pandemie', 'rki', 'inzidenz', 'neuinfektionen', 'impfpflicht', 'gesundheitsminister', 'impfung', 'welle', 'maskenpflicht', 'covid']
Looking at energiekrise with terms ['gas', 'habeck', 'öl', 'energie', 'energiekrise', 'wirtschaftsminister', 'akw', 'gazprom', 'atomkraft', 'strom', 'erdgas', 'pipeline', 'gaslieferungen', 'uniper', 'embargo', 'inflation', 'preise', 'wirtschaft', 'teurer', 'energiepreise', 'energie', 'kosten', 'verbraucher']
Looking at kulturUnterhaltung with terms ['film', 'serie', 'star', 'schauspieler', 'netflix', 'tatort', 'regisseur', 'album', 'schauspielerin', 'leutedestages', 'oscar', 'rtl', 'band', 'musik', 'sängerin', 'kino', 'fans', 'konzert', 'geschichte', 'buch', 'roman', 'gespräch', 'autor', 'kunst', 'schriftsteller', 'ausstellung', 'holocaust', 'historiker']
Looking at naherOstenNordafrika with terms ['türkei', 'iran', 'israel', 'afghanistan', 'taliban', 'mali', 'erdogan', 'protes

In [70]:
# ureg.groupby('user').sum

In [71]:
ureg.groupby('user').verkehrspolitik.sum().sort_values(ascending=False)

user
RND_de            635
derspiegel        559
welt              394
SZ                373
faznet            363
SZ_TopNews        357
zeitonline        345
ntvde             298
tagesschau        289
sternde           252
focusonline       234
ZDFheute          173
tazgezwitscher    154
taz_news          132
BILD              118
NZZ                97
DIEZEIT            92
fr                 69
phoenix_de         63
dpa                58
jungewelt          53
NZZde              44
SPIEGEL_Top        32
ardmoma            21
ntv_EIL            13
tagesschau_eil     10
tagesthemen         9
heutejournal        5
FAZ_Eil             5
SPIEGEL_EIL         5
ARD_Presseclub      5
WELTAMSONNTAG       2
reuters_de          0
tagesschau24        0
FOCUS_TopNews       0
BILD_News           0
Name: verkehrspolitik, dtype: int64

In [72]:
overall_ureg = ureg.groupby('user').sum()

/tmp/ipykernel_16490/2509313614.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  overall_ureg = ureg.groupby('user').sum()


In [73]:
ureg_classified = ureg
ureg_classified.to_csv('categorizedTweetsUreg.csv')

In [74]:
ureg_classified

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,politikEuropa,politikInternational,politikWahlen,socialMediaInternet,streiksDemos,tagespolitikDeutschland,ukrainekrieg,umweltKlima,verkehrspolitik,wirtschaft
0,2022-10-28 08:28:39+00:00,1585911408906366977,Das ganze Interview mit Markus Söder (CSU) gib...,30,4,34,1,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,0,0
1,2022-10-28 07:07:29+00:00,1585890981483671553,Vor Beginn des #CSU-Parteitags hat der bayrisc...,90,7,104,7,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,NaN,...,0,0,1,0,0,1,0,0,0,0
2,2022-10-28 06:31:09+00:00,1585881839570403329,@andreaslahmewd @bengtbergt @markus_soeder @bu...,1,0,1,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2022-10-28 06:08:32+00:00,1585876146125934593,Immer wieder gibt es #Luftalarm in der ukraini...,2,2,9,1,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
4,2022-10-28 05:02:44+00:00,1585859588506656768,"@unattached_man: Vielen Dank, haben wir an Sve...",0,0,0,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303568,2022-01-01 06:37:08+00:00,1477166986593505282,Mitten in der vierten Welle der Pandemie und m...,3,3,12,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
303569,2022-01-01 06:34:59+00:00,1477166443963863043,"""#Corona brachte uns etwas bei: das Aufeinande...",0,2,6,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
303570,2022-01-01 06:34:24+00:00,1477166296458579970,"Einheitstag, Weihnachten, erster Mai: Weil vie...",5,3,29,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
303571,2022-01-01 06:32:07+00:00,1477165724846198788,Wie können die nur! #Impfgegner und #Klimakris...,7,0,4,1,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
